In [1]:
#import tensorflow as tf
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import inspect as inspect
import cv2


In [2]:
# Define the paths to the train and test directories
root = os.getcwd() 
print(root)
train_data = os.path.join(root, r'dataset_Train.csv')
print(train_data)
test_data = os.path.join(root, r'dataset_Test.csv')
print(test_data)
#label_file = os.path.join(root, r'archive\TrainIJCNN2013\TrainIJCNN2013\gt.txt')
#print(label_file)


C:\Users\gtsrb\Desktop\detector
C:\Users\gtsrb\Desktop\detector\dataset_Train.csv
C:\Users\gtsrb\Desktop\detector\dataset_Test.csv


In [3]:
# Load the images and labels into arrays
X_train = []
y_train = []
train_data = pd.read_csv(train_data)
for i, row in train_data.iterrows():
    image = Image.open(row['Path'])
    #image = image.resize((32, 32))  # Resize the image to 32x32 pixels
    X_train.append(np.array(image))
    y_train.append(row['Bonding Boxs'])
X_train = np.array(X_train)
y_train = np.array(y_train)

#print(str(np.size(X_train)) + ' ' + str(X_train[0])) # size = 2780928000
#print(str(np.size(y_train)) + ' ' + str(y_train[0])) # size = 852

X_test = []
y_test = []
test_data = pd.read_csv(test_data)
for i, row in test_data.iterrows():
    image = Image.open(row['Path'])
    # image = image.resize((32, 32))  # Resize the image to 32x32 pixels
    X_test.append(np.array(image))
    y_test.append(row['Bonding Boxs'])
X_test = np.array(X_test)
y_test = np.array(y_test)

#print(str(np.size(X_test)) + ' ' + str(X_test[0])) # size = 1178304000
#print(str(np.size(y_test)) + ' ' + str(y_test[0])) # size = 361

In [4]:
# Preprocess the data
X_train = X_train.astype('float32') / 255  # Normalize pixel values to be between 0 and 1
X_test = X_test.astype('float32') / 255

y_train =  np.array([list(map(np.float32, s.split(';'))) for s in y_train])
y_test =  np.array([list(map(np.float32, s.split(';'))) for s in y_test])

#print(str(np.size(X_train)) + ' ' + str(X_train[0]))
#print(str(np.size(y_train)) + ' ' + str(y_train[:][:]))
#print(str(np.size(X_test)) + ' ' + str(X_test[0]))
#print(str(np.size(y_test)) + ' ' + str(y_test[:][:]))

In [5]:
# Convert the data to PyTorch tensors
X_train = torch.tensor(X_train.transpose((0, 3, 1, 2)))  # Transpose to match PyTorch's tensor format (NCHW)
y_train = torch.tensor(y_train).unsqueeze(-1)
X_test = torch.tensor(X_test.transpose((0, 3, 1, 2)))  # Transpose to match PyTorch's tensor format (NCHW)
y_test = torch.tensor(y_test).unsqueeze(-1)

#print(str(X_train.size()) + ' ' + str(X_train[0][0]))
#print(str(np.size(y_train)) + ' ' + str(y_train[:][:]))
#print(str(X_test.size()) + ' ' + str(X_test[0][0]))
#print(str(np.size(y_test)) + ' ' + str(y_test[:][:]))

In [6]:
class CustomConvLayer(nn.Module):
    def __init__(self, min_kernel_size):
        super(CustomConvLayer, self).__init__()
        self.min_kernel_size = min_kernel_size
        self.conv = nn.Conv2d(3, 3, kernel_size=1)

    def forward(self, x, bbox_size):
        print(bbox_size)
        min_bbox_size = bbox_size.min()
        kernel_size = max(self.min_kernel_size, min_bbox_size.item())
        print(kernel_size)
        padding = (kernel_size -1) // 2
        conv = nn.functional.conv2d(x, self.conv.weight, stride=1, padding=padding)
        return conv(x)
    

In [7]:
# # Define the model architecture
# class TrafficSignDetector(nn.Module):
#     def __init__(self):
#         super(TrafficSignDetector, self).__init__()
#         self.costum_conv = CustomConvLayer(min_kernel_size = 9)
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         #self.fc1 = nn.Linear(64 * 8 * 8, 64)
#         self.fc2_presence = nn.Linear(64, 1)  # Output for presence
#         self.fc2_bbox = nn.Linear(64, 4)  # Output for bounding box coordinates

#     def forward(self, x, bounding_boxes):
#         x = self.costum_conv(x, calculate_bounding_box_size(bounding_boxes))
#         x = self.pool(torch.relu(self.conv1(x)))
#         x = self.pool(torch.relu(self.conv2(x)))
#         print('before: ' + str(x.size()))
#         #x = x.reshape(-1, 64 * 8 * 8)
#         print('after: ' + str(x.size()))
#         #x = torch.relu(self.fc1(x))
#         presence = torch.sigmoid(self.fc2_presence(x))
#         bbox = self.fc2_bbox(x)
#         return presence, bbox

In [8]:
class TrafficSignDetector(nn.Module):
    def __init__(self, min_kernel_size):
        super(TrafficSignDetector, self).__init__()
        #self.custom_conv = CustomConvLayer(min_kernel_size)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(8, 2)
        self.fc1 = nn.Linear(256 * 94 * 164, 512)
        self.fc2 = nn.Linear(512, 5)  # Output: (confidence, x, y, width, height)

    def forward(self, x, bbox):
        #x = self.custom_conv(x, calculate_bounding_box_size(bbox_size))  # Apply custom convolutional layer
        x = self.pool(F.relu(self.conv1(x)))
        bbox = bbox.unsqueeze(-1)
        bbox = bbox.expand(-1, -1, x.size(2), x.size(3))
        x = torch.cat((x, bbox), dim=1)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.flatten(start_dim=1) # flatten, nur batch size ganz vorne beibehalten
        #x = x.view(-1, 256 * 8 * 8)  # Flatten before fully connected layers
        #x = x.reshape(-1, 256 * 8 * 8)  # Flatten before fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
def calculate_bounding_box_size(bounding_boxes):
    box_sizes = []
#    for bbox in bounding_boxes:
#         x_min = bbox[0]
#         y_min = bbox[1]
#         x_max = bbox[2]
#         y_max = bbox[3] 
#         width = x_max - x_min
#         height = y_max - y_min
#         box
    box_sizes = bounding_boxes[:, 2:] - bounding_boxes[:, :2]
    box_widths = box_sizes[:, 0]
    box_heights = box_sizes[:, 1]
    print("Widths:", box_widths)
    print("Heights:", box_heights)
    
    all_sizes = torch.cat((box_sizes[:, 0].unsqueeze(1), box_sizes[:, 1].unsqueeze(1)), dim=1)
    print(all_sizes)
        
    return all_sizes

In [10]:
# Create an instance of the model
#model = TrafficSignDetector()


In [11]:
# Initialize your model
min_kernel_size = 3  # You can adjust this as needed
model = TrafficSignDetector(min_kernel_size)

# Define your loss function and optimizer
criterion = nn.MSELoss()  # Use Mean Squared Error loss since it's a regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5  
batch_size = 64  
for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        batch_x = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        
        optimizer.zero_grad()
        #shape(batch_y) = torch.Size([64, 4])
        #shape(batch_x) = torch.Size([64, 3, 800, 1360])
        outputs = model(batch_x, batch_y)  
        loss = criterion(outputs, batch_y)  # Use the first element of bbox for training
        
        loss.backward()
        optimizer.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i//batch_size+1}/{len(X_train)//batch_size}], Loss: {loss.item():.4f}, Outputs: \n {outputs.item()}')

RuntimeError: Given groups=1, weight of size [128, 64, 3, 3], expected input[64, 68, 397, 677] to have 64 channels, but got 68 channels instead

In [ ]:
# Compile the model
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])


# Define the loss function and optimizer
# criterion_presence = nn.BCELoss()
# criterion_bbox = nn.SmoothL1Loss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
# num_batches = len(X_train) // 32
# num_epochs = 5

# for epoch in range(num_epochs):
#     total_loss = 0
#     for batch in range(num_batches):
#         batch_X = X_train[batch * 32 : (batch + 1) * 32]
#         batch_y = y_train[batch * 32 : (batch + 1) * 32]
        
#         print('batch_X: '+ str(batch_X.size()) + ' ' + str(batch_X.shape))
#         print('batch_y: '+ str(batch_y.size()) + ' ' + str(batch_y.shape))

#         optimizer.zero_grad()

#         # Forward pass
#         #outputs = model(batch_X, batch_y)

#         # Forward pass
#         presence_pred, bbox_pred = model(batch_X, batch_y)
        
#         # Compute the loss for presence prediction
#         presence_target = torch.tensor([[1.0] if bbox.any() else [0.0] for bbox in batch_y])
#         print('presence_target: '+ str(presence_target.size()) + ' ' + str(presence_target.shape))
#         presence_loss = criterion_presence(presence_pred, presence_target)

#           # Compute the loss for bounding box regression
#         bbox_loss = criterion_bbox(bbox_pred, batch_y)

#         # Combine the losses
#         loss = presence_loss + bbox_loss
#         total_loss += loss.item()

        
#         # Compute the loss
#         #batch_y = batch_y.long()
#         #loss = criterion(outputs, batch_y)
#         #total_loss += loss.item()

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#     # Print average loss for the epoch
#     average_loss = total_loss / num_batches
#     print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}")

In [ ]:
torch.save(model,r'.\cache02.h5')

In [ ]:
model = torch.load(r'.\cache.h5')

total_correct = 0
total_samples = 0

model.eval()
batch_size = 1
with torch.no_grad():
    for batch in range(0, len(X_test)):
        batch_x = X_test[batch:batch + batch_size]
        batch_y = y_test[batch:batch + batch_size]
        
        print(str(batch) + ': ' + str(batch_y))
        
        outputs = model(batch_x, batch_y)
        x_min = str(outputs[:, 1].item() * (100 - 0) + 0)
        y_min = str(outputs[:, 2].item() * (100 - 0) + 0)
        x_max = str(outputs[:, 1].item() + outputs[:, 3].item())
        y_max = str(outputs[:, 2].item() + outputs[:, 4].item())
        print(str(batch) + ': ' + str(outputs[:,0]) + '; ' + x_min + ', ' + y_min + ', '  + x_max + ', ' + y_max )
      
    

In [ ]:
def load_vid(path):
    video = cv2.VideoCapture(path)
    
    if not video.isOpened():
        print("Fehler beim laden des Videos: " + path)
        return []
    
    frames = []
    frame_count = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Save frames for processing
        frames.append(frame)
        frame_count += 1

        #cv2.imshow('Video' , frame)

        #if cv2.waitKey(1) & 0xFF == ord('q'):
            #break

    video.release()
    #cv2.destroyAllWindows()
    
    return frames


In [ ]:
video_path_list = []
frames_lists = []

for name, _, datas in os.walk(r'C:\Users\gtsrb\Desktop\detector\videos'):
    for data in datas:
        video_path = os.path.join(name, data)

        if any(video_path.endswith(ext) for ext in ['.mp4', '.avi','.mkv']):
            video_path_list.append(video_path)

for video_path in video_path_list:
   frames_lists.append(load_vid(video_path))
   #print(video_list[len(video_list)-1])'


In [ ]:
print(len(video_path_list))

In [ ]:
# preprocessing
frames_lists = np.array(frames_lists)
frames_lists = frames_lists.astype('float32')/255
frames_lists = torch.tensor(frames_lists.transpose((0, 3, 1, 2)))

In [ ]:

model = torch.load(r'.\cache.h5')

In [ ]:
num_batches = len(frames_lists) // 32
threshold=0.5 

frame_count = 0
for batch in range(0, len(frames_lists), 64):
    batch_frames = frames_lists[batch: batch + 64]
    
    optimizer.zero_grad

    outputs = model(batch_frames)

    predictions = outputs.squeeze()
    traffic_sign_present = predictions >= threshold
    
    if traffic_sign_present.any():
         current_frame = batch_frames[0].numpy()
         frame_path = r"./cache/frame_" + str(frame_count) + str(output[:, 4]) + ".jpg"
         cv2.imwrite(frame_path, current_frame)
         print("Traffic sign detected in the current frame. Saved as frame_" + str(frame_count) + str(output[:, 4]) + ".jpg")
         frame_count += 1

    loss = criterion(outputs)
    total_loss += loss.item()

    loss.backward()
    optimizer.step()